# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0     puerto natales -51.7236  -72.4875     12.25        47   
1        1         olyokminsk  60.3736  120.4264      8.04        93   
2        2              vadso  70.0744   29.7487      9.49        93   
3        3         georgetown   5.4112  100.3354     26.96        92   
4        4  port-aux-francais -49.3500   70.2167      2.08        81   

   Cloudiness  Wind Speed Country        Date  
0         100       12.35      CL  1695748096  
1          26        0.60      RU  1695748096  
2          75        1.54      NO  1695748096  
3          20        1.54      MY  1695747834  
4          70       16.24      TF  1695748097

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display
# Configure the map plot
humidity = city_data_df["Humidity"]

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo= True, 
    tiles="OSM",
    frame_width= 800,
    frame_height = 600,
    size= "Humidity",
    color = "City",
    alpha=.75)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# max temp 20 degrees, higher than 15, windspeed less than 2.5 m/s, cloudiness more than 15%
narrowed_df = city_data_df.loc[(city_data_df["Max Temp"] > 15) & (city_data_df["Max Temp"] < 20)
                              & (city_data_df["Wind Speed"] < 2.5) & (city_data_df["Cloudiness"] > 15)].reset_index(drop=True)

narrowed_df.head()

City_ID            City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0       86          nemuro  43.3236  145.5750     16.25        90          79   
1       91          albany  42.6001  -73.9662     18.43        68         100   
2      115        namasuba   0.6894   32.4214     18.74        91          88   
3      152  yuzhno-kurilsk  44.0306  145.8556     15.47        94          89   
4      169            aksu  41.1231   80.2644     18.23        31          94   

   Wind Speed Country        Date  
0        1.23      JP  1695748119  
1        1.79      US  1695747832  
2        0.79      UG  1695748126  
3        1.94      RU  1695748135  
4        1.66      CN  1695748140

In [14]:
# Drop any rows with null values
narrowed_df.isnull().sum()
#narrowed_df = narrowed_df.dropna()
#no null values to drop!

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [15]:
clean_df = narrowed_df.loc[:,~(narrowed_df.isnull().any())]
clean_df.reset_index(drop=True)
# Display sample data
clean_df.head()

City_ID            City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0       86          nemuro  43.3236  145.5750     16.25        90          79   
1       91          albany  42.6001  -73.9662     18.43        68         100   
2      115        namasuba   0.6894   32.4214     18.74        91          88   
3      152  yuzhno-kurilsk  44.0306  145.8556     15.47        94          89   
4      169            aksu  41.1231   80.2644     18.23        31          94   

   Wind Speed Country        Date  
0        1.23      JP  1695748119  
1        1.79      US  1695747832  
2        0.79      UG  1695748126  
3        1.94      RU  1695748135  
4        1.66      CN  1695748140

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity  \
0                        nemuro      JP  43.3236  145.5750        90   
1                        albany      US  42.6001  -73.9662        68   
2                      namasuba      UG   0.6894   32.4214        91   
3                yuzhno-kurilsk      RU  44.0306  145.8556        94   
4                          aksu      CN  41.1231   80.2644        31   
5                       malango      ID  -2.9627  119.9001        94   
6                        yoichi      JP  43.2039  140.7703        83   
7                       guozhen      CN  34.3659  107.3590        91   
8                          kone      NC -21.0595  164.8658        78   
9                         tippi      ET   7.2000   35.4500        97   
10                    ulladulla      AU -35.3500  150.4667        93   
11  laikit, laikit ii (dimembe)      ID   1.4792  124.9735        94   
12                        yauya      PE  -8.9833  -77.3000        38   
13       happy valley-goose bay      CA  53.3168  -60.3315        47   
14                   long jetty      AU -33.3667  151.4833        91   
15                      fortuna      US  40.5982 -124.1573        90   
16                    zaozhuang      CN  34.8647  117.5542        86   
17                       shache      CN  38.4167   77.2406        21   
18                      potenza      IT  40.6443   15.8086        79   
19                pollock pines      US  38.7613 -120.5866        67   
20                        sinop      TR  41.7711   34.8709        88   
21                      kashgar      CN  39.4547   75.9797        23   
22                        dunay      RU  42.8803  132.3375        89   
23                        hotan      CN  37.0997   79.9269        19   
24                    mont-dore      NC -22.2833  166.5833        67   
25                        korla      CN  41.7597   86.1469        45   

   Hotel Name  
0              
1              
2              
3              
4              
5              
6              
7              
8              
9              
10             
11             
12             
13             
14             
15             
16             
17             
18             
19             
20             
21             
22             
23             
24             
25

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
lat = ""
lon = ""
categories= "accommodation.hotel"
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
limit = 5

params = {
    "categories":categories,
    "filter": filters,
    "bias": bias,
    "limit": limit,
    "lang": "en",
    "apiKey": geoapify_key
}

In [18]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df    

Starting hotel search
nemuro - nearest hotel: イーストハーバーホテル
albany - nearest hotel: No hotel found
namasuba - nearest hotel: No hotel found
yuzhno-kurilsk - nearest hotel: Dom Druzhby
aksu - nearest hotel: Akesu Tianyuan International Hotel
malango - nearest hotel: Madarana
yoichi - nearest hotel: エーヴランド ホテル
guozhen - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
tippi - nearest hotel: Tepi genet penision
ulladulla - nearest hotel: Hotel Marlin
laikit, laikit ii (dimembe) - nearest hotel: Chandra Lestari Hotel Cottage & Restoran
yauya - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
long jetty - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
zaozhuang - nearest hotel: 枣庄大酒店
shache - nearest hotel: 莎車新盛賓舘
potenza - nearest hotel: Tourist Hotel
pollock pines - nearest hotel: North Canyon Inn Bed & Breakfast
sinop - nearest hotel: No hotel found
kashgar - nearest hotel: 深航国际酒店
dunay -

City Country      Lat       Lng  Humidity  \
0                        nemuro      JP  43.3236  145.5750        90   
1                        albany      US  42.6001  -73.9662        68   
2                      namasuba      UG   0.6894   32.4214        91   
3                yuzhno-kurilsk      RU  44.0306  145.8556        94   
4                          aksu      CN  41.1231   80.2644        31   
5                       malango      ID  -2.9627  119.9001        94   
6                        yoichi      JP  43.2039  140.7703        83   
7                       guozhen      CN  34.3659  107.3590        91   
8                          kone      NC -21.0595  164.8658        78   
9                         tippi      ET   7.2000   35.4500        97   
10                    ulladulla      AU -35.3500  150.4667        93   
11  laikit, laikit ii (dimembe)      ID   1.4792  124.9735        94   
12                        yauya      PE  -8.9833  -77.3000        38   
13       happy valley-goose bay      CA  53.3168  -60.3315        47   
14                   long jetty      AU -33.3667  151.4833        91   
15                      fortuna      US  40.5982 -124.1573        90   
16                    zaozhuang      CN  34.8647  117.5542        86   
17                       shache      CN  38.4167   77.2406        21   
18                      potenza      IT  40.6443   15.8086        79   
19                pollock pines      US  38.7613 -120.5866        67   
20                        sinop      TR  41.7711   34.8709        88   
21                      kashgar      CN  39.4547   75.9797        23   
22                        dunay      RU  42.8803  132.3375        89   
23                        hotan      CN  37.0997   79.9269        19   
24                    mont-dore      NC -22.2833  166.5833        67   
25                        korla      CN  41.7597   86.1469        45   

                                         Hotel Name  
0                                       イーストハーバーホテル  
1                                    No hotel found  
2                                    No hotel found  
3                                       Dom Druzhby  
4                Akesu Tianyuan International Hotel  
5                                          Madarana  
6                                        エーヴランド ホテル  
7                                    No hotel found  
8                                 Pacifik Appartels  
9                               Tepi genet penision  
10                                     Hotel Marlin  
11         Chandra Lestari Hotel Cottage & Restoran  
12                                   No hotel found  
13                                  Hotel North Two  
14                                   No hotel found  
15             Comfort Inn & Suites Redwood Country  
16                                            枣庄大酒店  
17                                           莎車新盛賓舘  
18                                    Tourist Hotel  
19                 North Canyon Inn Bed & Breakfast  
20                                   No hotel found  
21                                           深航国际酒店  
22                                   No hotel found  
23  Hua Yi International Hotel (accepts foreigners)  
24                                Les Cases de Plum  
25                                Silver Star Hotel

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo= True, 
    tiles="OSM",
    frame_width= 800,
    frame_height = 600,
    size= "Humidity",
    color = "City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)